In [1]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
gpa = pd.read_csv("../data/GPA.csv")

In [3]:
gpa.head()

,Calculation1,Campus,City,County/State/Country,Fall Term,Level,School,Adm GPA (SUM),App GPA (SUM),Enrl GPA (SUM)
0,GERMAN SWISS INTERNATIONAL SCH670905,Berkeley,NaN,HONG KONG,2012,Freshman,GERMAN SWISS INTERNATIONAL SCH,NaN,3.604000,NaN
1,AQUINAS HIGH SCHOOL052772,Riverside,San Bernardino,San Bernardino,1995,Freshman,AQUINAS HIGH SCHOOL,NaN,3.361429,NaN
2,DESIGN SCIENCE HIGH SCHOOL054003,Los Angeles,Fresno,Fresno,2012,Freshman,DESIGN SCIENCE HIGH SCHOOL,NaN,3.818000,NaN
3,BLOOMFIELD HILLS HIGH SCHOOL232447,Universitywide,Bloomfield Twp,Michigan,2015,Freshman,BLOOMFIELD HILLS HIGH SCHOOL,3.942,3.950000,NaN
4,A B MILLER HIGH SCHOOL050944,Santa Barbara,Fontana,San Bernardino,1998,Freshman,A B MILLER HIGH SCHOOL,NaN,3.518571,NaN


In [4]:
transfers = pd.read_csv("../data/Transfers.csv")

In [5]:
transfers.head()

,Calculation1,Campus,City,County/State/ Territory,Fall Term,School,Sch Src TR,Uad Uc Ethn 6 Cat,adm (SUM),app (SUM),enr (SUM)
0,BUTTE COLLEGE4226,Davis,Oroville,Butte,2007,BUTTE COLLEGE,California Community Colleges (CCC),White,22.0,32,4.0
1,IMPERIAL VALLEY COLLEGE4358,Riverside,Imperial,Imperial,2008,IMPERIAL VALLEY COLLEGE,California Community Colleges (CCC),Hispanic/ Latino,13.0,27,3.0
2,ALLAN HANCOCK COLLEGE4002,Universitywide,Santa Maria,Santa Barbara,2012,ALLAN HANCOCK COLLEGE,California Community Colleges (CCC),White,97.0,133,65.0
3,AMERICAN RIVER COLLEGE4004,Berkeley,Sacramento,Sacramento,1996,AMERICAN RIVER COLLEGE,California Community Colleges (CCC),Hispanic/ Latino,8.0,14,6.0
4,CUESTA COLLEGE4101,Santa Barbara,San Luis Obispo,San Luis Obispo,2013,CUESTA COLLEGE,California Community Colleges (CCC),All,122.0,262,42.0


In [6]:
freshmen = pd.read_csv("../data/Freshmen.csv")

In [7]:
freshmen.head()

,Calculation1,Campus,City,County/State/ Territory,Fall Term,School,Sch Src FR,Uad Uc Ethn 6 Cat,adm (SUM),app (SUM),enr (SUM)
0,F BRAVO MEDICAL MAGNET HS51719,Riverside,Los Angeles,Los Angeles,2008,F BRAVO MEDICAL MAGNET HS,California Public High School,Hispanic/ Latino,39.0,50,10.0
1,ARAGON HIGH SCHOOL53163,Santa Barbara,San Mateo,San Mateo,2012,ARAGON HIGH SCHOOL,California Public High School,Asian,28.0,53,4.0
2,ETIWANDA HIGH SCHOOL50888,San Diego,Etiwanda,San Bernardino,2012,ETIWANDA HIGH SCHOOL,California Public High School,Asian,11.0,34,4.0
3,UPLAND HIGH SCHOOL53586,Merced,Upland,San Bernardino,2005,UPLAND HIGH SCHOOL,California Public High School,Asian,6.0,7,NaN
4,MORNINGSIDE HIGH SCHOOL51262,Los Angeles,Inglewood,Los Angeles,2006,MORNINGSIDE HIGH SCHOOL,California Public High School,All,5.0,22,NaN


In [8]:
freshmen.shape

(587501, 11)

# Initial Exploratory Tasks

1. How many high schools are in each of the datasets? How many years do each of the datasets cover?
2. How many high schools are included in all three of the datasets? How many high schools are included in only 2 of the datasets? How many high schools are included in only one of the datasets?
3. What high schools have the highest number of freshmen admitted? What high schools schools have the highest number of transfer students admitted? How does this differ across UC campuses?
4. What high schools have the highest freshman admit rate? What schools have the highest freshmen transfer rate? How would would you even calculate these things? :o How does this differ across UC campuses?
5. What California counties are most represented in the data? What non-California states are most represented in the data? What other countries are most represented in the data? How does this differ across UC campuses?
6. Of the high schools that have students admitted, which high school has the highest ratio of admitted GPA to enrolled GPA? Of the high schools that have students enrolled, which high school has the highest ratio of enrolled GPA to admitted GPA? Is there a significant (linear) correlation between high school GPA and acceptance into a particular UC campus?
7. What is the relationship between admit rate and admit GPA? (Answer through a visualization) How is this different between the relationship between enrollment rate and enrollment GPA? How does this differ across each UC Campus?
8. How different is the admit rate between different ethnicities? How does this relate to the enrollment rate between different ethnicities? How does this vary between freshmen and transfer students? How does this vary across different campuses?